In [ ]:
from azure.search.documents.indexes.models import SearchIndex
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SimpleField,
    SemanticSettings,
    PrioritizedFields,
    SearchFieldDataType,
    CorsOptions, 
    SearchIndex,
    SemanticConfiguration,   
    SemanticField,
    VectorSearch,
    VectorSearchProfile,
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters  
)
import os

# Load .env file
load_dotenv()

# Get settings from .env
search_service_name = f"https://{os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')}.search.windows.net/"
search_service_api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
index_name = os.getenv("INDEX_NAME")

# Create a client
credential = AzureKeyCredential(search_service_api_key)
client = SearchIndexClient(endpoint=f"{search_service_name}", credential=credential)

In [ ]:
# This configures the vector search algorithm. The parameters are set to the defaults for the algorithm.
vector_search = VectorSearch(
    algorithms=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            parameters=HnswParameters(
                m=4,
                ef_construction=int(400),
                ef_search=int(400),
                metric="cosine",
            ),
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-search-profile", algorithm="my-vector-config"
        )
    ],
)

semantic_config = SemanticConfiguration(
    name="semantic-config",
    prioritized_fields=PrioritizedFields(
        prioritized_content_fields=[
            SemanticField(field_name="content"),
            SemanticField(field_name="summary")
        ]                   
    ),
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []

# Define the index
index = SearchIndex(
    name=index_name,
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),       
        SimpleField(name="title", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="content", type=SearchFieldDataType.String, filterable=True, sortable=True, searchable=True),
        SearchField(name="summary", type=SearchFieldDataType.String, filterable=True, sortable=True, searchable=True),
        SimpleField(name="source", type=SearchFieldDataType.String, searchable=True),
        SimpleField(name="page", type=SearchFieldDataType.Int32, searchable=True),        
        SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True, vector_search_dimensions=int(1536), vector_search_profile="my-vector-search-profile"),
        SearchField(name="summary_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True, vector_search_dimensions=int(1536), vector_search_profile="my-vector-search-profile")
    ],
    vector_search=vector_search,
    semantic_settings=semantic_settings,
    scoring_profiles=scoring_profiles,
    cors_options=cors_options,
)

client.create_or_update_index(index)
print(f"Index {index_name} created.")